In [ ]:
import re
import gc
import os
import sys
import emoji
import swifter
import warnings
import unicodedata
import pandas as pd
import numpy as np

from numba import cuda
from llama_cpp import Llama
from bertopic import BERTopic
from cuml.manifold import UMAP
from timeout_decorator import timeout
from cuml.cluster import HDBSCAN, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.metrics.pairwise import cosine_similarity
from cuml.metrics.cluster import silhouette_score as silhouette_cuml

#Importações para o Llama 2
from huggingface_hub import hf_hub_download
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain_community.llms import LlamaCpp
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

##%load_ext cudf.pandas
import pandas as pd
import torch
import transformers
import re

In [ ]:
pd.set_option('display.max_colwidth', None)

# Suprimir todos os warnings do Pandas
warnings.filterwarnings('ignore', category=pd.errors.DtypeWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, message="No sentence-transformers model found")

# Restaurar as configurações de aviso padrão (opcional)
warnings.filterwarnings('default')

## Funções de pré-processamento dos textos

In [ ]:
# Função para remover menções a usuários (@algumusuario)
def remove_mentions(text):
    return re.sub(r'@[\w_]+', '', text)


# Função para remover acentos repetidos (!!!, ???, etc)
def remove_repeated_punctuation(text):
    return re.sub(r'([!?.])\1+', r'\1', text)


# Função para remover emojis
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data).strip()


# Função para capitalizar as palavras (início de frase maisculo, restante minusculo)
def capitalize_text(text):
    #Strip
    text = text.strip()
    # Dividir o texto em sentenças
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Capitalizar a primeira letra de cada sentença
    capitalized_sentences = [sentence.capitalize() for sentence in sentences]
    # Juntar as sentenças novamente em um único texto
    return ' '.join(capitalized_sentences)


# Função para remover hiperlinks.
def remove_links_and_line_breaks(text):
    # Remove hiperlinks
    text = re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)
    # Substitui quebras de linha por espaços
    text = re.sub(r'\n\d*', ' ', text)
    # Substitui quebras de linha no meio de palavras por um espaço
    text = re.sub(r'(\w+)\n(\w+)', r'\1 \2', text)
    return text.strip()


# Função para remover caracteres especiais e substituir emojis pelas suas palavras.
def replace_special_characters(text):
    # Converte emojis de números em seus valores numéricos
    text = emoji.demojize(text)
    # Define um dicionário com a correspondência entre emojis e números
    emoji_to_text = {
        ":keycap_0:": "0",
        ":keycap_1:": "1",
        ":keycap_2:": "2",
        ":keycap_3:": "3",
        ":keycap_4:": "4",
        ":keycap_5:": "5",
        ":keycap_6:": "6",
        ":keycap_7:": "7",
        ":keycap_8:": "8",
        ":keycap_9:": "9",
        ":squid:":"Lula",
        ":brazil:":"Brasil"
    }
    # Substitui os emojis de números pelos valores numéricos
    for emoji_text, number in emoji_to_text.items():
        text = text.replace(emoji_text, number)
    return emoji.emojize(text)


# Função para remover hashtags.
def remove_hashtags(text):
    # Use a expressão regular para encontrar e remover hashtags
    text_without_hashtags = re.sub(r'#\w+', '', text)
    return text_without_hashtags.strip()

In [ ]:
## Obtenção dos tweets de 8 a 15 de jan de 2023

In [ ]:
ano = '2023'
mes = '01'
tipo = 'tweets'

## Leitura Tweets Bolsonaro

In [ ]:
query = 'query_bolsonaro'
data_list = []

for dia in range(8, 13):
    dia_str = f"{dia:02d}"
    diretorio = f'./corpus/{query}/{tipo}-{query}-{ano}_{mes}_{dia_str}.parquet'

    print(diretorio)
    # Tenta ler o arquivo parquet, se existir
    try:
        data = pd.read_parquet(diretorio)
        data_list.append(data)
    except FileNotFoundError:
        print(f"Arquivo não encontrado para o dia {dia}")

df_bolsonaro = pd.concat(data_list, ignore_index=True)
df_bolsonaro = df_bolsonaro[['id', 'text', 'created_at', 'hashtags']]

## Leitura Tweets Lula

In [ ]:
query='query_lula'
data_list = []

for dia in range(8, 13):
    dia_str = f"{dia:02d}"
    diretorio = f'./corpus/{query}/{tipo}-{query}-{ano}_{mes}_{dia_str}.parquet'

    print(diretorio)
    # Tenta ler o arquivo parquet, se existir
    try:
        data = pd.read_parquet(diretorio)
        data_list.append(data)
    except FileNotFoundError:
        print(f"Arquivo não encontrado para o dia {dia}")

df_lula = pd.concat(data_list, ignore_index=True)
df_lula = df_lula[['id', 'text', 'created_at', 'hashtags']][:

## Leitura Tweets Ato Golpista

In [ ]:
query='atos_golpistas'
data_list = []

for dia in range(8, 11):
    dia_str = f"{dia:02d}"
    diretorio = f'./corpus/dados/{tipo}-{query}/{query}/{tipo}-{query}-{ano}_{mes}_{dia_str}.parquet'

    print(diretorio)
    # Tenta ler o arquivo parquet, se existir
    try:
        data = pd.read_parquet(diretorio)
        data_list.append(data)
    except FileNotFoundError:
        print(f"Arquivo não encontrado para o dia {dia}")

df_atos_golpistas = pd.concat(data_list, ignore_index=True)
df_atos_golpistas = df_atos_golpistas[['id', 'text', 'created_at', 'hashtags']]

In [ ]:
# Aplicar as funções aos dados da coluna "text"

def tratamento_base(df):
    df['text'] = df['text'].apply(remove_mentions)
    df['text'] = df['text'].apply(remove_repeated_punctuation)
    df['text'] = df['text'].apply(remove_links_and_line_breaks)
    df['text'] = df['text'].apply(replace_special_characters)
    df['text'] = df['text'].apply(remove_emojis)
    df['text'] = df['text'].apply(remove_hashtags)
    df['text'] = df['text'].apply(capitalize_text)
    
    # Remover linhas com valores em branco na coluna "text"
    df = df[df['text'].notna() & df['text'] != '']
    
    # Remover linhas duplicadas com base na coluna "text"
    df = df.drop_duplicates(subset='text')

    return df

In [ ]:
def tratamento_base(df):
    df['text'] = df['text'].swifter.apply(remove_mentions)
    df['text'] = df['text'].swifter.apply(remove_repeated_punctuation)
    df['text'] = df['text'].swifter.apply(remove_links_and_line_breaks)
    df['text'] = df['text'].swifter.apply(replace_special_characters)
    df['text'] = df['text'].swifter.apply(remove_emojis)
    df['text'] = df['text'].swifter.apply(remove_hashtags)
    df['text'] = df['text'].swifter.apply(capitalize_text)
    # Remover linhas com valores em branco na coluna "text"
    df = df[df['text'].notna() & df['text'] != '']
    # Remover linhas duplicadas com base na coluna "text"
    df = df.drop_duplicates(subset='text')
    return df

In [ ]:
df_bolsonaro = tratamento_base(df_bolsonaro)
df_lula = tratamento_base(df_lula)
df_atos_golpistas = tratamento_base(df_atos_golpistas)

In [ ]:
df_atos_golpistas = tratamento_base(df_atos_golpistas)

## Geração das embeddings das sentenças

In [ ]:
import torch
from sentence_transformers import SentenceTransformer

In [ ]:
def embeddings_generation(df):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    #Seleção do modelo bertimbau para a geração das embeddings
    model = SentenceTransformer('neuralmind/bert-large-portuguese-cased', device = device)
    
    sentences = df['text']
    sentences = sentences.tolist()

    # Geração de embeddings de texto
    embeddings = model.encode(sentences, show_progress_bar=True, device=device)

    del(model)
    
    #Normalização das embeddings
    normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1)[:, np.newaxis]

    # Redução de dimensionalidade com UMAP
    umap_embeddings = UMAP(n_neighbors=10,  # Reduzido para 10 vizinhos
                                n_components=512,
                                metric='cosine').fit_transform(normalized_embeddings)
    
    #df['embedding_default'] = embeddings.tolist()
    df['embedding_normalized'] = normalized_embeddings.tolist()
    df['embedding'] = umap_embeddings.tolist()

    return df

In [ ]:
df_bolsonaro = embeddings_generation(df_bolsonaro)
df_bolsonaro.to_parquet('df_bolsonaro_processed.parquet', index=False)
del(df_bolsonaro)

df_lula = embeddings_generation(df_lula)
df_lula.to_parquet('df_lula_processed.parquet', index=False)
del(df_lula)

df_atos_golpistas = embeddings_generation(df_atos_golpistas)
df_atos_golpistas.to_parquet('df_atos_golpistas_processed.parquet', index=False)
del(df_atos_golpistas)

## Obtenção das melhores métricas do HDBSCan

In [ ]:
import datetime

def get_cluster_results_metrics(df):
    # Lista para armazenar os resultados
    resultados = []

    embeddings = np.array(df['embedding'].tolist())
    
    # Iterando sobre os parâmetros
    for min_cluster_size in range(10, 201, 10):
        for metric in ['euclidean']:
            for cluster_selection_method in ['leaf', 'eom']:
                print("Início: ", datetime.datetime.now())
                # Clustering com HDBSCAN
                cluster = HDBSCAN(min_cluster_size=min_cluster_size,
                                  metric=metric,
                                  cluster_selection_method=cluster_selection_method).fit(embeddings)
    
                # Verificar se há pelo menos dois rótulos únicos
                unique_labels = np.unique(cluster.labels_)
                print("Gerou o cluster: ", datetime.datetime.now())
                if len(unique_labels) > 1:
                    # Calcular as métricas de avaliação
                    labels = cluster.labels_

                    cluster=None
                    del(cluster)
                    
                    try:
                        print("Silhouette CUML")
                        silhouette_avg = silhouette_cuml(X=embeddings, labels = labels, metric='cosine')
                    except Exception:
                        print("Silhouette CPU")
                        silhouette_avg = silhouette_score(X=embeddings, labels = labels, metric='cosine')

                    # Armazenar os resultados
                    resultados.append({
                        'min_cluster_size': min_cluster_size,
                        'metric': metric,
                        'cluster_selection_method': cluster_selection_method,
                        'silhouette_score': silhouette_avg,
                        'labels': len(unique_labels)
                    })
    
                    # Imprimir os resultados de cada rodada
                    print(f"Parâmetros: min_cluster_size={min_cluster_size}, metric={metric}, cluster_selection_method={cluster_selection_method}")
                    print(f"Silhueta: {silhouette_avg}")
                    print(f"Labels: {len(unique_labels)}")

                    print("Fim: ", datetime.datetime.now())
                    print("-" * 30)

    return resultados

In [ ]:
def clustering(df, melhor_resultado):

    print("Início: ", datetime.datetime.now())
    
    embeddings = np.array(df['embedding'].tolist())
    
    # Clustering com HDBSCAN - Pulando a redução de dimensionalidade por enquanto
    cluster = HDBSCAN(min_cluster_size=melhor_resultado['min_cluster_size'],      # Aumentado para 20
                          metric=melhor_resultado['metric'],       # Alterado para distância euclideana
                          cluster_selection_method=melhor_resultado['cluster_selection_method']).fit(embeddings)

    import matplotlib.pyplot as plt
    # Preparação dos dados
    umap_data = UMAP(n_neighbors=10, n_components=2, metric='cosine').fit_transform(embeddings)
    result = pd.DataFrame(umap_data, columns=['x', 'y'])
    result['labels'] = cluster.labels_
    
    # Visualização dos clusters
    fig, ax = plt.subplots(figsize=(20, 10))
    outliers = result.loc[result.labels == -1, :]
    clustered = result.loc[result.labels != -1, :]
    plt.scatter(outliers.x, outliers.y, color='#BDBDBD', s=0.05)
    plt.scatter(clustered.x, clustered.y, c=clustered.labels, s=0.05, cmap='hsv_r')
    plt.colorbar()

    df['label'] = cluster.labels_

    print("Fim: ", datetime.datetime.now())

    return df

In [ ]:
df_bolsonaro_processed = pd.read_parquet('df_bolsonaro_processed.parquet')
resultados_bolsonaro = get_cluster_results_metrics(df_bolsonaro_processed)

with open('./results/clustering/resultados_cluster_bolsonaro.txt', 'w') as arquivo:
    arquivo.write('\n'.join(map(str, resultados_bolsonaro)))

melhor_resultado_bolsonaro = max(resultados_bolsonaro, key=lambda x: x['silhouette_score'])

In [ ]:
df_lula_processed = pd.read_parquet('df_lula_processed.parquet').iloc[:500000]
resultados_lula = get_cluster_results_metrics(df_lula_processed)

with open('./results/clustering/resultados_cluster_lula.txt', 'w') as arquivo:
    arquivo.write('\n'.join(map(str, resultados_lula)))

melhor_resultado_lula = max(resultados_lula, key=lambda x: x['silhouette_score'])

In [ ]:
df_atos_golpistas_processed = pd.read_parquet('df_atos_golpistas_processed.parquet').iloc[:500000]
resultados_atos_golpistas = get_cluster_results_metrics(df_atos_golpistas_processed)

with open('./results/clustering/resultados_cluster_atos_golpistas.txt', 'w') as arquivo:
    arquivo.write('\n'.join(map(str, resultados_atos_golpistas)))

melhor_resultado_atos_golpistas = max(resultados_atos_golpistas, key=lambda x: x['silhouette_score'])

## Extração de Tópicos - BERTopic

In [ ]:
def generate_topics_bertopic(df):
    embeddings = np.array(df['embedding_normalized'].tolist())
    docs = df['text'].tolist()

    hdbscan_model = HDBSCAN(min_samples=10, gen_min_span_tree=True, cluster_selection_method='eom', prediction_data=True)
    umap_model = UMAP(n_neighbors=15, n_components=512, min_dist=0.0, metric='cosine', random_state=42)

    topic_model = BERTopic(hdbscan_model=hdbscan_model, umap_model=umap_model)

    topics, probs = topic_model.fit_transform(docs, embeddings)

    topic_info = topic_model.get_topic_info()

    del hdbscan_model, umap_model, topic_model

    return topics, probs, topic_info

In [ ]:
df_bolsonaro_processed = pd.read_parquet('df_bolsonaro_processed.parquet')
topics, probs, topic_info = generate_topics_bertopic(df_bolsonaro_processed)

df_bolsonaro_processed['topics'] = topics

df_bolsonaro_processed.to_parquet('df_bolsonaro_topic.parquet', index=False)
topic_info.to_parquet('topicos_bolsonaro.parquet', index=False)

In [ ]:
df_lula_processed = pd.read_parquet('df_lula_processed.parquet')
topics, probs, topic_info = generate_topics_bertopic(df_lula_processed)

df_lula_processed['topics'] = topics

df_lula_processed.to_parquet('df_lula_topic.parquet', index=False)
topic_info.to_parquet('topicos_lula.parquet', index=False)

In [ ]:
df_atos_golpistas_processed = pd.read_parquet('df_atos_golpistas_processed.parquet')
topics, probs, topic_info = generate_topics_bertopic(df_atos_golpistas_processed)

df_atos_golpistas_processed['topics'] = topics

df_atos_golpistas_processed.to_parquet('df_atos_golpistas_topic.parquet', index=False)
topic_info.to_parquet('topicos_atos_golpistas.parquet', index=False)

## Análise descritiva simples dos dados finais

In [ ]:
df_bolsonaro_topic = pd.read_parquet('df_bolsonaro_topic.parquet')
df_bolsonaro_topic.drop(df_bolsonaro_topic[df_bolsonaro_topic.topics == -1].index, inplace=True)

In [ ]:
df_lula_topic = pd.read_parquet('df_lula_topic.parquet')
df_lula_topic.drop(df_lula_topic[df_lula_topic.topics == -1].index, inplace=True)

In [ ]:
df_atos_golpistas_topic = pd.read_parquet('df_atos_golpistas_topic.parquet')
df_atos_golpistas_topic.drop(df_atos_golpistas_topic[df_atos_golpistas_topic.topics == -1].index, inplace=True)

In [ ]:
# Calculando as estatísticas
topic_counts = df_bolsonaro_topic['topics'].value_counts()
max_topic = topic_counts.idxmax()
min_topic = topic_counts.idxmin()
mean_topic = topic_counts.mean()
median_topic = topic_counts.median()

summary = {
    "Topic with Max Entries": max_topic,
    "Max Entries": topic_counts[max_topic],
    "Topic with Min Entries": min_topic,
    "Min Entries": topic_counts[min_topic],
    "Mean of Entries": mean_topic,
    "Median of Entries": median_topic
}

# Exibindo o resumo
summary_df = pd.DataFrame([summary])
summary_df

# 925 tópicos

In [ ]:
# Calculando as estatísticas
topic_counts = df_lula_topic['topics'].value_counts()
max_topic = topic_counts.idxmax()
min_topic = topic_counts.idxmin()
mean_topic = topic_counts.mean()
median_topic = topic_counts.median()

summary = {
    "Topic with Max Entries": max_topic,
    "Max Entries": topic_counts[max_topic],
    "Topic with Min Entries": min_topic,
    "Min Entries": topic_counts[min_topic],
    "Mean of Entries": mean_topic,
    "Median of Entries": median_topic
}

# Exibindo o resumo
summary_df = pd.DataFrame([summary])
summary_df

# 1306 tópicos

In [ ]:
# Calculando as estatísticas
topic_counts = df_atos_golpistas_topic['topics'].value_counts()
max_topic = topic_counts.idxmax()
min_topic = topic_counts.idxmin()
mean_topic = topic_counts.mean()
median_topic = topic_counts.median()

summary = {
    "Topic with Max Entries": max_topic,
    "Max Entries": topic_counts[max_topic],
    "Topic with Min Entries": min_topic,
    "Min Entries": topic_counts[min_topic],
    "Mean of Entries": mean_topic,
    "Median of Entries": median_topic
}

# Exibindo o resumo
summary_df = pd.DataFrame([summary])
summary_df

# 1646 tópicos

In [ ]:
def filter_top_twenty_topics(df):
    top_ten = df['topics'].value_counts().nlargest(20).to_dict()
    
    topten_list = []
    for key, value in top_ten.items():
        topten_list.append(key)
        
    return df[df['topics'].isin(topten_list)].reset_index(drop=True)


In [ ]:
filter_top_twenty_topics_df_atos_golpistas = filter_top_twenty_topics(df_atos_golpistas_topic)
filter_top_twenty_topics_df_atos_golpistas.to_csv('filter_top_twenty_topics_df_atod_golpistas.csv', index=False)

In [ ]:
filter_top_twenty_topics_df_lula = filter_top_twenty_topics(df_lula_topic)
filter_top_twenty_topics_df_lula.to_csv('filter_top_twenty_topics_df_lula.csv', index=False)

In [ ]:
filter_top_twenty_topics_df_bolsonaro = filter_top_twenty_topics(df_bolsonaro_topic)
filter_top_twenty_topics_df_bolsonaro.to_csv('filter_top_twenty_topics_df_bolsonaro.csv', index=False)

## Sumarização dos tweets usando Llama 2 - 13B

In [ ]:
#Download do modelo
def download_hf_model(repo_id, filename):
    downloaded_model_path = hf_hub_download(
        repo_id=repo_id,
        filename=filename,
        use_auth_token=True)

    return downloaded_model_path

In [ ]:
# Função do llama com prompt para a sumarização
@timeout(75)
def llama_summarizer(llm, tweets, model):
    tweets = ";".join(tweets).replace('\n', '')

    if model == 'llama':
        template = """[INST] <<SYS>> I am providing you with an 'Input' of a set of texts in Brazilian Portuguese that are separated by ";" between them. Provide as 'Output' a summary in continuous text with your own words also in Brazilian Portuguese,
        without repeating the original texts and without citing examples, covering the main subjects being mentioned in the input texts briefly and in a general way. Do not form your own opinions; all content in the summarization must be based on the 
        content found in the texts. When citing a trend or any behavior, use something like 'Os autores citaram que (...)' <<SYS>>
    
        Input: '''{tweets}''' [/INST]
        
        Output:
        """
    elif model == 'bode':
        template = """[INST] <<SYS>> Estou fornecendo a você uma 'Entrada' de um conjunto de textos em português do Brasil que estão separados por ";" entre eles. Forneça como 'Saída' um resumo em texto contínuo com suas próprias palavras também em português do Brasil,
        sem repetir os textos originais e sem citar exemplos, abrangendo os principais temas mencionados nos textos de entrada de forma breve e geral. Não forme suas próprias opiniões; todo o conteúdo na sumarização deve ser baseado no
        conteúdo encontrado nos textos. Ao citar uma tendência ou comportamento, use algo como 'Os autores citaram que (...).' <<SYS>>

        Input: "{tweets}" [/INST]
        
        Output:
        """
        
    prompt = PromptTemplate(input_variables=["tweets"], template=template)
    runnable = prompt | llm | StrOutputParser()
    response = runnable.invoke({"tweets": tweets})

    response = response.replace("[]", "")

    return response

@timeout(75)
def llama_translate(llm, text):

    template = """[INST] <<SYS>> Traduza o texto abaixo para português do Brasil. Caso ele já esteja em Português, apenas repita ele como saída. <<SYS>>

    Input: "{text}" [/INST]
    
    Output:
        """
        
    prompt = PromptTemplate(input_variables=["tweets"], template=template)
    runnable = prompt | llm | StrOutputParser()
    response = runnable.invoke({"text": text})

    response = response.replace("[]", "")

    return response

In [ ]:
def get_llama_llm(model_repo_path, model_filename):
    ### Llama-CPP
    callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
    n_gpu_layers = -1
    n_batch = 768

    model_path_hf = download_hf_model(model_repo_path, model_filename)

    #Instanciação do Lllama
    llm = LlamaCpp(
        model_path=model_path_hf,
        n_gpu_layers=n_gpu_layers,
        n_batch=n_batch,
        temperature = 0.3,
        max_tokens = 768,
        n_ctx = 8192,
        top_p=1,
        n_threads=8,
        callback_manager=callback_manager,
        verbose=True
    )

    return llm

In [ ]:
def define_model(model_type):
    if model_type=='llama':
        llm = get_llama_llm(model_repo_path='TheBloke/Llama-2-13B-chat-GGUF', model_filename='llama-2-13b-chat.Q8_0.gguf')
    elif model_type=='bode':
        llm = get_llama_llm(model_repo_path='recogna-nlp/bode-13b-alpaca-pt-br-gguf', model_filename='bode-13b-alpaca-q8_0.gguf')
    elif model_type=='mistral':
        llm = get_llama_llm(model_repo_path='TheBloke/Mistral-7B-Instruct-v0.2-GGUF', model_filename='mistral-7b-instruct-v0.2.Q8_0.gguf')
    
    return llm

## Sumarização dos tweets usando o Llama 3 8B

In [ ]:
def get_pipeline_llama3(model_id):
    pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
    )

    return pipeline

In [ ]:
def llama_3_summarizer(pipeline, tweets):
    tweets = ";".join(tweets).replace('\n', '')

    messages = [
    {"role": "system", "content": """I am providing you with an 'Input' of a set of texts in Brazilian Portuguese that are separated by ";" between them. Provide as 'Output' a summary in continuous text with your own words also in Brazilian Portuguese, without repeating the original texts and without citing examples, covering the main subjects being mentioned in the input texts briefly and in a general way. Do not form your own opinions; all content in the summarization must be based on the content found in the texts. When citing a trend or any behavior, use something like 'Os autores citaram que (...)'"""},
    {"role": "user", "content": f"Input: {tweets}"},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
    prompt,
    max_new_tokens=768,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.3,
    top_p=1,
    )

    print(outputs[0]["generated_text"][len(prompt):])
    return outputs[0]["generated_text"][len(prompt):]


def llama_3_translate(pipeline, text):

    messages = [
    {"role": "system", "content": """Traduza o texto dado como "Input" para português do Brasil. Caso ele já esteja em Português, apenas repita ele como saída."""},
    {"role": "user", "content": f"Input: {text}"},
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
    prompt,
    max_new_tokens=768,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.3,
    top_p=1,
    )

    return outputs[0]["generated_text"][len(prompt):]

## Pipeline de sumarização Multinível

In [ ]:
# Define a função auxiliar para agrupar os tweets até atingir um limite de palavras
def agrupar_tweets(tweets, limite_palavras=1536):
    grupos = []
    grupo_atual = []
    palavras_no_grupo = 0
    for tweet in tweets:
        palavras_tweet = len(tweet.split())
        if palavras_no_grupo + palavras_tweet <= limite_palavras:
            grupo_atual.append(tweet)
            palavras_no_grupo += palavras_tweet
        else:
            grupos.append(grupo_atual)
            grupo_atual = [tweet]
            palavras_no_grupo = palavras_tweet
    if grupo_atual:
        grupos.append(grupo_atual)
    return grupos

# Lógica principal para sumarizar os tweets de forma recursiva
def sumarizar_tweets(tweets, model, llm=None, pipeline=None):
    while len(tweets) > 1:
        grupos = agrupar_tweets(tweets)
        sumarios = []
        for grupo in grupos:
            try:
                if model not in ('llama_3', 'llama_3.1'):
                    sumario = llama_summarizer(llm=llm, tweets=grupo, model=model)
                    sumarios.append(sumario)
                else:
                    sumario = llama_3_summarizer(pipeline=pipeline, tweets=grupo)
                    sumarios.append(sumario)
            except:
                print("\n\nTimeout de execução no LLama. Continuando com os próximos grupos.")
                continue
            
        tweets = sumarios
        print("Tamanho: ", len(tweets))
    return tweets[0]

In [ ]:
def get_summaries(df, model, llm=None, pipeline=None):
    # Dicionário para armazenar os resultados
    sumarios = {}
    
    # Itera sobre os diferentes tópicos
    for topic in df['topics'].unique():
        subset_df = df[df['topics'] == topic]
        
        # Converte a coluna "text" para lista
        tweets = subset_df['text'].tolist()

        # Aplica a sumarização
        resultado = sumarizar_tweets(tweets, model, llm, pipeline)
        
        # Armazena o resultado no dicionário
        sumarios[topic] = resultado

    return sumarios

### Base atos golpistas

In [ ]:
modelos = ['llama', 'mistral', 'bode', 'llama_3', 'llama_3.1']

for model in modelos:
    pipeline = None
    llm = None
    
    if model not in ('llama_3', 'llama_3.1'):
        #Limpeza de VRAM
        try:
            del llm
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
            
        llm = define_model(model)
    else:
        try:
            del pipeline
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
        if model == 'llama_3':
            model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
        elif model == 'llama_3.1':
            model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        pipeline = get_pipeline_llama3(model_id)

    sumarios_atos_golpistas = get_summaries(filter_top_twenty_topics_df_atos_golpistas, model, llm, pipeline)

    for chave, mensagem in sumarios_atos_golpistas.items():
        if model not in ('llama_3', 'llama_3.1'):
            mensagem = llama_translate(llm, mensagem)
        else:
            mensagem = llama_3_translate(pipeline, mensagem)
        sumarios_atos_golpistas[chave] = mensagem

    with open(f'./results/topics/resultados_topics_atos_golpistas_{model}.txt', 'w') as arquivo:
        for chave, mensagem in sumarios_atos_golpistas.items():
            arquivo.write(f"Tópico {chave}:\n{mensagem}\n\n")

### Base Lula

In [ ]:
modelos = ['llama', 'mistral', 'bode', 'llama_3', 'llama_3.1']

#Limpeza da VRAM
device = cuda.get_current_device()
device.reset()

for model in modelos:
    pipeline = None
    llm = None
    
    if model not in ('llama_3', 'llama_3.1'):
        try:
            del llm
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
        llm = define_model(model)
    else:
        try:
            del pipeline
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
        if model == 'llama_3':
            model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
        elif model == 'llama_3.1':
            model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        pipeline = get_pipeline_llama3(model_id)

    sumarios_lula = get_summaries(filter_top_twenty_topics_df_lula, model, llm, pipeline)
    
    for chave, mensagem in sumarios_lula.items():
        if model not in ('llama_3', 'llama_3.1'):
            mensagem = llama_translate(llm, mensagem)
        else:
            mensagem = llama_3_translate(pipeline, mensagem)
        sumarios_lula[chave] = mensagem
    
    with open(f'./results/topics/resultados_topics_lula_{model}.txt', 'w') as arquivo:
        for chave, mensagem in sumarios_lula.items():
            arquivo.write(f"Tópico {chave}:\n{mensagem}\n\n")

### Base Bolsonaro

In [ ]:
modelos = ['llama', 'mistral', 'bode', 'llama_3', 'llama_3.1']

for model in modelos:
    pipeline = None
    llm = None
    
    if model not in ('llama_3', 'llama_3.1'):
        try:
            del llm
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
        llm = define_model(model)
    else:
        try:
            del pipeline
            gc.collect()
            torch.cuda.empty_cache()
        except:
            continue
        if model == 'llama_3':
            model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
        elif model == 'llama_3.1':
            model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
        pipeline = get_pipeline_llama3(model_id)

    sumarios_bolsonaro = get_summaries(filter_top_twenty_topics_df_bolsonaro, model, llm, pipeline)
    
    for chave, mensagem in sumarios_bolsonaro.items():
        if model not in ('llama_3', 'llama_3.1'):
            mensagem = llama_translate(llm, mensagem)
        else:
            mensagem = llama_3_translate(pipeline, mensagem)
        sumarios_bolsonaro[chave] = mensagem
    
    with open(f'./results/topics/resultados_topics_bolsonaro_{model}.txt', 'w') as arquivo:
        for chave, mensagem in sumarios_bolsonaro.items():
            arquivo.write(f"Tópico {chave}:\n{mensagem}\n\n")

In [ ]:
try:
    del llm
    gc.collect()
    torch.cuda.empty_cache()
except:
    print("Variável não existe")

try:
    del pipeline
    gc.collect()
    torch.cuda.empty_cache()
except:
    print("Variável não existe")

## Validações utilizando o BERTScore e BERTScore-p

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM, BertModel
from bert_score import BERTScorer
import gc

In [ ]:
def open_summary_topics(caminho_arquivo):
    try:
        # Abrir o arquivo em modo de leitura com a codificação UTF-8
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            # Ler o conteúdo do arquivo
            linhas = arquivo.readlines()

            # Inicializar um dicionário vazio
            dicionario = {}
            chave_atual = None
            valor_atual = ""

            # Iterar sobre as linhas do arquivo
            for linha in linhas:
                # Remover espaços em branco extras no início e no final da linha
                linha = linha.strip()
                if linha.startswith("Tópico"):
                    # Se a linha começar com "Tópico", atualize a chave atual
                    if chave_atual is not None:
                        # Se já tiver uma chave atual, armazene o valor atual no dicionário
                        dicionario[chave_atual] = valor_atual
                    chave_atual = linha
                    valor_atual = ""
                else:
                    # Se a linha não começar com "Tópico", adicione-a ao valor atual
                    valor_atual += linha + "\n"

            # Armazene o último valor atual no dicionário
            dicionario[chave_atual] = valor_atual

        return dicionario
    except FileNotFoundError:
        print("Arquivo não encontrado.")
    except Exception as e:
        print("Ocorreu um erro ao processar o arquivo:", e)

In [ ]:
def dataframe_para_string(df, topico):
    # Filtrar o DataFrame pelo tópico especificado
    df_filtrado = df[df['topics'] == int(topico)]
    
    # Concatenar os dados da coluna 'text' em uma única string
    texto_completo = ' '.join(df_filtrado['text'])
    
    return texto_completo

In [ ]:
def validate_bertscore_old(reference, candidate):
    # BERTScore calculation
    scorer = BERTScorer(model_type='microsoft/deberta-xlarge-mnli')
    P, R, F1 = scorer.score([candidate], [reference])
    
    # Armazenar os resultados em um dicionário
    result_dict = {
        "BERTScore Precision": P.mean(),
        "BERTScore Recall": R.mean(),
        "BERTScore F1": F1.mean()
    }

    del scorer
    gc.collect()
    
    return result_dict

In [ ]:
def validate_bertscore(reference, candidate, segment_size=995, beta = 2):
    from bert_score import BERTScorer
    import gc
    import torch
    
    def split_text(text, size):
        """
        Divide o texto em segmentos de até 'size' caracteres,
        garantindo que não corte palavras no meio.
        """
        segments = []
        start = 0
        while start < len(text):
            end = start + size
            # Ajusta o final para o último espaço antes do limite
            if end < len(text):
                while end > start and text[end] not in [' ', '\n', '.', ',']:
                    end -= 1
            if end == start:  # Caso não encontre delimitador válido
                end = start + size
            segments.append(text[start:end].strip())
            start = end
        return segments
    
    # Inicializa o scorer com o modelo desejado
    scorer = BERTScorer(model_type='neuralmind/bert-large-portuguese-cased', num_layers=24)
    
    # Divide os textos em segmentos menores
    reference_segments = split_text(reference, segment_size)
    candidate_segments = split_text(candidate, segment_size)

    weight = (len(candidate) ** beta) / ((len(candidate) ** beta) + len(reference))

    P_list, R_list, F1_list = [], [], []

    # Compara todos os segmentos do candidato com todos os da referência
    for cand_seg in candidate_segments:
        cand_len = len(cand_seg.split())
        for ref_seg in reference_segments:
            ref_len = len(ref_seg.split())
            P, R, F1 = scorer.score([cand_seg], [ref_seg])
            P_list.append(P.mean().item())
            R_list.append(R.mean().item())
            F1_list.append(F1.mean().item())

    # Calcula a média dos Top-50 valores
    def top_k_avg(scores, k=20):
        if scores:
            top_k_scores = sorted(scores, reverse=True)[:k]
            return sum(top_k_scores) / len(top_k_scores)
        return 0

    # Calcula a média e o máximo dos resultados
    result_dict = {
        "Weight": weight,
        "Weighted F1 Avg": ((sum(F1_list) / len(F1_list)) * weight) * 0.3 + (sum(F1_list) / len(F1_list)) * 0.7 if F1_list else 0,
        "Weighted F1 Max": (max(F1_list) * weight) * 0.3 + max(F1_list) * 0.7 if F1_list else 0,
        "Weighted F1 Min": (min(F1_list) * weight) * 0.3 + min(F1_list) * 0.7 if F1_list else 0,
        "Weighted F1 Top-k Avg": (top_k_avg(F1_list) * weight) * 0.3 + top_k_avg(F1_list) * 0.7,
        "Precision Avg": sum(P_list) / len(P_list) if P_list else 0,
        "Recall Avg": sum(R_list) / len(R_list) if R_list else 0,
        "F1 Avg": sum(F1_list) / len(F1_list) if F1_list else 0,
        "Precision Max": max(P_list) if P_list else 0,
        "Recall Max": max(R_list) if R_list else 0,
        "F1 Max": max(F1_list) if F1_list else 0,
        "Precision Min": min(P_list) if P_list else 0,
        "Recall Min": min(R_list) if R_list else 0,
        "F1 Min": min(F1_list) if F1_list else 0,
        "Precision Top-k Avg": top_k_avg(P_list),
        "Recall Top-k Avg": top_k_avg(R_list),
        "F1 Top-k Avg": top_k_avg(F1_list),
    }
    
    # Limpeza de memória
    del scorer
    gc.collect()
    torch.cuda.empty_cache()
    
    return result_dict

In [ ]:
summary_bolsonaro_llama = open_summary_topics('./results/topics/resultados_topics_bolsonaro_llama.txt')
summary_lula_llama = open_summary_topics('./results/topics/resultados_topics_lula_llama.txt')
summary_atos_golpistas_llama = open_summary_topics('./results/topics/resultados_topics_atos_golpistas_llama.txt')

summary_bolsonaro_mistral = open_summary_topics('./results/topics/resultados_topics_bolsonaro_mistral.txt')
summary_lula_mistral = open_summary_topics('./results/topics/resultados_topics_lula_mistral.txt')
summary_atos_golpistas_mistral = open_summary_topics('./results/topics/resultados_topics_atos_golpistas_mistral.txt')

summary_bolsonaro_bode = open_summary_topics('./results/topics/resultados_topics_bolsonaro_bode.txt')
summary_lula_bode = open_summary_topics('./results/topics/resultados_topics_lula_bode.txt')
summary_atos_golpistas_bode = open_summary_topics('./results/topics/resultados_topics_atos_golpistas_bode.txt')

summary_bolsonaro_llama_3 = open_summary_topics('./results/topics/resultados_topics_bolsonaro_llama_3.txt')
summary_lula_llama_3 = open_summary_topics('./results/topics/resultados_topics_lula_llama_3.txt')
summary_atos_golpistas_llama_3 = open_summary_topics('./results/topics/resultados_topics_atos_golpistas_llama_3.txt')

summary_bolsonaro_llama_3_1 = open_summary_topics('./results/topics/resultados_topics_bolsonaro_llama_3.1.txt')
summary_lula_llama_3_1 = open_summary_topics('./results/topics/resultados_topics_lula_llama_3.1.txt')
summary_atos_golpistas_llama_3_1 = open_summary_topics('./results/topics/resultados_topics_atos_golpistas_llama_3.1.txt')

In [ ]:
palavras_llama2 = []

for i, topico in enumerate(summary_atos_golpistas_llama):
    palavras = summary_atos_golpistas_llama[topico].split()
    palavras_llama2.append(len(palavras))

soma_llama2 = sum(palavras_llama2)
tamanho_llama2 = len(palavras_llama2)

media_llama2 = soma_llama2/tamanho_llama2

print("Média Llama 2: ", media_llama2)


for i, topico in enumerate(summary_atos_golpistas_llama_3):
    palavras = summary_atos_golpistas_llama_3[topico].split()
    palavras_llama2.append(len(palavras))

soma_llama2 = sum(palavras_llama2)
tamanho_llama2 = len(palavras_llama2)

media_llama2 = soma_llama2/tamanho_llama2

print("Média Llama 3: ", media_llama2)

for i, topico in enumerate(summary_atos_golpistas_mistral):
    palavras = summary_atos_golpistas_mistral[topico].split()
    palavras_llama2.append(len(palavras))

soma_llama2 = sum(palavras_llama2)
tamanho_llama2 = len(palavras_llama2)

media_llama2 = soma_llama2/tamanho_llama2

print("Média Mistral: ", media_llama2)

for i, topico in enumerate(summary_atos_golpistas_bode):
    palavras = summary_atos_golpistas_bode[topico].split()
    palavras_llama2.append(len(palavras))

soma_llama2 = sum(palavras_llama2)
tamanho_llama2 = len(palavras_llama2)

media_llama2 = soma_llama2/tamanho_llama2

print("Média Bode: ", media_llama2)



In [ ]:
#### result_bolsonaro = {}

print("Sumários Bolsonaro")

print("BERTSCore Llama")
for chave, valor in summary_bolsonaro_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_bolsonaro[f'BertScore Bolsonaro - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_bolsonaro_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_bolsonaro[f'BertScore Bolsonaro - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_bolsonaro_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_bolsonaro[f'BertScore Bolsonaro - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_bolsonaro_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_bolsonaro[f'BertScore Bolsonaro - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_bolsonaro_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_bolsonaro[f'BertScore Bolsonaro - Llama 3.1 - Tópico {numero_topico}: '] = result

with open('./results/topics/result_bertscore_bolsonaro_v6.txt', 'w') as file:
    for key, value in result_bolsonaro.items():
        file.write(f"{key}:\n")
        for sub_key, sub_value in value.items():
            file.write(f"\t{sub_key}: {sub_value}\n")
        file.write("\n")

In [ ]:
print("Sumários Lula")

result_lula = {}

print("BERTSCore Llama")
for chave, valor in summary_lula_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_lula[f'BertScore Lula - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_lula_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_lula[f'BertScore Lula - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_lula_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_lula[f'BertScore Lula - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_lula_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_lula[f'BertScore Lula - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_lula_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_lula[f'BertScore Lula - Llama 3.1 - Tópico {numero_topico}: '] = result


with open('./results/topics/result_bertscore_lula_v6.txt', 'w') as file:
    for key, value in result_lula.items():
        file.write(f"{key}:\n")
        for sub_key, sub_value in value.items():
            file.write(f"\t{sub_key}: {sub_value}\n")
        file.write("\n")

In [ ]:
print("Sumários Atos Golpistas")

result_atos_golpistas = {}

print("BERTSCore Llama")
for chave, valor in summary_atos_golpistas_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas[f'BertScore Atos Golpistas - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_atos_golpistas_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)

    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas[f'BertScore Atos Golpistas - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_atos_golpistas_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)

    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas[f'BertScore Atos Golpistas - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_atos_golpistas_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas[f'BertScore Atos Golpistas - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_atos_golpistas_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas[f'BertScore Atos Golpistas - Llama 3.1 - Tópico {numero_topico}: '] = result

with open('./results/topics/result_bertscore_atos_golpistas_v6.txt', 'w') as file:
    for key, value in result_atos_golpistas.items():
        file.write(f"{key}:\n")
        for sub_key, sub_value in value.items():
            file.write(f"\t{sub_key}: {sub_value}\n")
        file.write("\n")

## Validações utilizando o Rouge Score

In [ ]:
from rouge_score.rouge_scorer import RougeScorer
scorer = RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [ ]:
result_bolsonaro_rouge = {}

print("Sumários Bolsonaro")

print("BERTSCore Llama")
for chave, valor in summary_bolsonaro_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_bolsonaro_rouge[f'Rouge Score - Bolsonaro - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_bolsonaro_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_bolsonaro_rouge[f'Rouge Score - Bolsonaro - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_bolsonaro_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_bolsonaro_rouge[f'Rouge Score - Bolsonaro - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_bolsonaro_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_bolsonaro_rouge[f'Rouge Score - Bolsonaro - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_bolsonaro_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_bolsonaro, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_bolsonaro_rouge[f'Rouge Score - Bolsonaro - Llama 3.1 - Tópico {numero_topico}: '] = result

with open('./results/topics/result_bertscore_bolsonaro_rougescore.txt', 'w') as file:
    for key, value in result_bolsonaro_rouge.items():
        file.write(f"\t{key} {value}\n")
        file.write("\n")

In [ ]:
print("Sumários Lula")

result_lula_rouge = {}

print("BERTSCore Llama")
for chave, valor in summary_lula_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_lula_rouge[f'Rouge Score - Lula - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_lula_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_lula_rouge[f'Rouge Score - Lula - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_lula_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_lula_rouge[f'Rouge Score - Lula - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_lula_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_lula_rouge[f'Rouge Score - Lula - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_lula_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_lula, numero_topico)
    result = scorer.score(texto_completo, valor).items()

    result_lula_rouge[f'Rouge Score - Lula - Llama 3.1 - Tópico {numero_topico}: '] = result


with open('./results/topics/result_bertscore_lula_rougescore.txt', 'w') as file:
    for key, value in result_lula_rouge.items():
        file.write(f"\t{key} {value}\n")
        file.write("\n")

In [ ]:
print("Sumários Atos Golpistas")

result_atos_golpistas_rouge = {}

print("BERTSCore Llama")
for chave, valor in summary_atos_golpistas_llama.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas_rouge[f'Rouge Score - Atos Golpistas - Llama 2 - Tópico {numero_topico}: '] = result

print("BERTSCore Mistral")
for chave, valor in summary_atos_golpistas_mistral.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)

    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas_rouge[f'Rouge Score - Atos Golpistas - Mistral - Tópico {numero_topico}: '] = result

print("BERTSCore Bode")
for chave, valor in summary_atos_golpistas_bode.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)

    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas_rouge[f'Rouge Score - Atos Golpistas - Bode - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3")
for chave, valor in summary_atos_golpistas_llama_3.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas_rouge[f'Rouge Score - Atos Golpistas - Llama 3 - Tópico {numero_topico}: '] = result

print("BERTSCore Llama 3.1")
for chave, valor in summary_atos_golpistas_llama_3_1.items():
    numero_topico = chave.split()[1].replace(":", "")
    print("Tópico:", numero_topico)

    texto_completo = dataframe_para_string(filter_top_twenty_topics_df_atos_golpistas, numero_topico)
    result = validate_bertscore(texto_completo, valor)

    result_atos_golpistas_rouge[f'Rouge Score - Atos Golpistas - Llama 3.1 - Tópico {numero_topico}: '] = result

with open('./results/topics/result_bertscore_atos_golpistas_rougescore.txt', 'w') as file:
    for key, value in result_atos_golpistas_rouge.items():
        file.write(f"\t{key} {value}\n")
        file.write("\n")